<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install spacy==2.3.0
!pip3 install spacy_langdetect
!pip3 install tqdm
import spacy.cli
spacy.cli.download('en_core_web_lg')
spacy.cli.download('pl_core_news_lg')

In [14]:
import spacy
from spacy_langdetect import LanguageDetector
import en_core_web_lg
nlp = en_core_web_lg.load()
import pl_core_news_lg
nlp_pl = pl_core_news_lg.load()
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

from ipykernel import kernelapp as app
from tqdm import tqdm
import pandas as pd
import string

In [3]:
cat = pd.read_json('/content/drive/My Drive/Library_catalogue.jl', lines=True)

In [8]:
def detect_language(df):
    language = []
    for doc in tqdm(nlp.pipe(df['title'].values, batch_size=1000), desc='Detecting language', total=len(df)):
        if doc.is_parsed:
            if doc._.language['language'] == 'en': language.append('en')
            else: language.append('pl')
        else: language.append('')

    df['language'] = language
    return df

In [12]:
def preprocess(input_df):
    preprocessed_df = detect_language(input_df)

    for lang in ['pl', 'en']:
        if lang == 'en': model = nlp
        elif lang == 'pl': model = nlp_pl

        df = input_df[input_df['language'] == lang]
        stopwords = model.Defaults.stop_words
        preprocessed = []
        for doc in tqdm(model.pipe(df['title'].values, batch_size=200), desc=f'Preprocessing {lang}', total=len(df)):
            if doc.is_parsed:
                tokens = [token.lemma_.lower() for token in doc if (token.text.lower() not in stopwords and token.text not in string.punctuation)]
                if len(tokens) > 0: preprocessed.append(tokens)
                else: preprocessed.append('preprocessing_fail')
            else: preprocessed.append('preprocessing_fail')

        df['tokens'] = preprocessed
        df = df[df['tokens'] != 'preprocessing_fail']
        preprocessed_df = preprocessed_df.append(df)
    return preprocessed_df

In [ ]:
cat.sort_values(by='year', ascending=False, inplace=True)
cat.drop_duplicates(subset='title', inplace=True)
cat = preprocess(cat)
cat.to_json('/content/drive/My Drive/Library_catalogue_preprocessed.jl', lines=True, orient='records')